# Import Modules

In [1]:
import os
import sys
import pickle

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)

# #############################################################################
sys.path.insert(0, os.path.join(os.environ["PROJ_irox"], "data"))
from proj_data_irox import (
    bulk_dft_data_path, unique_ids_path,
    static_irox_structures_path)

# #############################################################################
from IPython.display import display

In [2]:
from out_data.inputs_nersc_iro2 import ignore_ids as ignore_ids_nersc_iro2
# from out_data.inputs_nersc_iro3 import ignore_ids as ignore_ids_nersc_iro3
ignore_ids_nersc_iro3 = []

from out_data.inputs_sher_iro2 import ignore_ids as ignore_ids_sher_iro2
from out_data.inputs_sher_iro3 import ignore_ids as ignore_ids_sher_iro3

from out_data.inputs_slac_iro2 import ignore_ids as ignore_ids_slac_iro2
# from out_data.inputs_slac_iro3 import ignore_ids as ignore_ids_slac_iro3
ignore_ids_slac_iro3 = []

ignore_ids_iro2 = ignore_ids_nersc_iro2 + ignore_ids_sher_iro2 + ignore_ids_slac_iro2
ignore_ids_iro3 = ignore_ids_nersc_iro3 + ignore_ids_sher_iro3 + ignore_ids_slac_iro3

print("len(ignore_ids_iro2):", len(ignore_ids_iro2))
print("len(ignore_ids_iro3):", len(ignore_ids_iro3))

ignore_ids_dict = {
    "AB2": ignore_ids_iro2,
    "AB3": ignore_ids_iro3}


# Pickling data ######################################################
import os; import pickle
directory = "out_data"
if not os.path.exists(directory): os.makedirs(directory)
with open(os.path.join(directory, "ignore_ids.pickle"), "wb") as fle:
    pickle.dump(ignore_ids_dict, fle)
# #####################################################################

len(ignore_ids_iro2): 94
len(ignore_ids_iro3): 0


In [3]:
from out_data.ids_to_run_nersc_iro2 import ids_to_run as ids_to_run_nersc_iro2
from out_data.ids_to_run_nersc_iro3 import ids_to_run as ids_to_run_nersc_iro3

from out_data.ids_to_run_slac_iro2 import ids_to_run as ids_to_run_slac_iro2
# from out_data.ids_to_run_slac_iro3 import ids_to_run as ids_to_run_slac_iro3
ids_to_run_slac_iro3 = []

from out_data.ids_to_run_sher_iro2 import ids_to_run as ids_to_run_sher_iro2
from out_data.ids_to_run_sher_iro3 import ids_to_run as ids_to_run_sher_iro3

ids_to_run_iro2 = ids_to_run_nersc_iro2 + ids_to_run_sher_iro2 + ids_to_run_slac_iro2
ids_to_run_iro3 = ids_to_run_nersc_iro3 + ids_to_run_sher_iro3 + ids_to_run_slac_iro3

ids_to_run_iro3 = list(set(ids_to_run_iro3))
ids_to_run_iro2 = list(set(ids_to_run_iro2))

ids_to_run_sher = ids_to_run_sher_iro2 + ids_to_run_sher_iro3
ids_to_run_nersc = ids_to_run_nersc_iro2 + ids_to_run_nersc_iro3
ids_to_run_slac = ids_to_run_slac_iro2 + ids_to_run_slac_iro3

ids_to_run_sher = list(set(ids_to_run_sher))
ids_to_run_nersc = list(set(ids_to_run_nersc))
ids_to_run_slac = list(set(ids_to_run_slac))

# Script Inputs

In [4]:
stoich_i = "AB2"

# Read Data

In [5]:
with open(bulk_dft_data_path, "rb") as fle:
    df_bulk_dft = pickle.load(fle)
    df_bulk_dft = df_bulk_dft[(df_bulk_dft["source"] == "raul")]

with open(static_irox_structures_path, "rb") as fle:
    df_static = pickle.load(fle)
    df_static = df_static[(df_static["source"] == "chris")]

df_ids = pd.read_csv(unique_ids_path)

with open("out_data/df_new_jobs.pickle", "rb") as fle:
    df_new_jobs = pickle.load(fle)

with open("out_data/df_irox_long.pickle", "rb") as fle:
    df_irox_long = pickle.load(fle)

In [6]:
unique_ids_in_process = df_new_jobs[df_new_jobs["stoich"] == stoich_i]["id"]
unique_ids_in_process = unique_ids_in_process.unique().tolist()
unique_ids_in_process = [i for i in unique_ids_in_process if i.isdigit()]

In [7]:
def method(row_i):
    atoms = row_i["atoms"]
    num_atoms = atoms.get_number_of_atoms()
    return(num_atoms)

df_static["num_atoms"] = df_static.apply(method, axis=1)
df_static = df_static.drop(["atoms", "path", "source",], axis=1)

In [8]:
def method(row_i):
    new_column_values_dict = {}

    stoich_i = row_i["stoich"]

    id_old = row_i["id_old"]
    id_old_str = str(id_old).zfill(3)

    # #########################################################################
    if row_i.name in df_bulk_dft.index:
        new_column_values_dict["done"] = True
    else:
        new_column_values_dict["done"] = False

    # #########################################################################
    if id_old_str in unique_ids_in_process:
        new_column_values_dict["being_processed"] = True
    else:
        new_column_values_dict["being_processed"] = False

    # #########################################################################
    if stoich_i == "AB2":
        ignore_ids_i = ignore_ids_iro2
    elif stoich_i == "AB3":
        ignore_ids_i = ignore_ids_iro3
    # id_old = str(row_i["id_old"]).zfill(3)

    if id_old_str in ignore_ids_i:
        new_column_values_dict["ignored"] = True
    else:
        new_column_values_dict["ignored"] = False

    # #########################################################################
    # stoich_i = row_i["stoich"] == "AB2"
    if stoich_i == "AB2":
        ids_to_run_i = ids_to_run_iro2
    elif stoich_i == "AB3":
        ids_to_run_i = ids_to_run_iro3

    if id_old in ids_to_run_i:
        new_column_values_dict["ids_to_run"] = True
    else:
        new_column_values_dict["ids_to_run"] = False


    # #########################################################################
    if stoich_i == "AB2":
        if id_old in ids_to_run_sher_iro2:
            new_column_values_dict["cluster"] = "sherlock"
        elif id_old in ids_to_run_nersc_iro2:
            new_column_values_dict["cluster"] = "nersc"
        elif id_old in ids_to_run_slac_iro2:
            new_column_values_dict["cluster"] = "slac"
        else:
            new_column_values_dict["cluster"] = np.nan

    elif stoich_i == "AB3":
        if id_old in ids_to_run_sher_iro3:
            new_column_values_dict["cluster"] = "sherlock"
        elif id_old in ids_to_run_nersc_iro3:
            new_column_values_dict["cluster"] = "nersc"
        elif id_old in ids_to_run_slac_iro3:
            new_column_values_dict["cluster"] = "slac"
        else:
            new_column_values_dict["cluster"] = np.nan


    # #########################################################################
    df_new_jobs_i = df_new_jobs[
        (df_new_jobs["stoich"] == stoich_i) & \
        (df_new_jobs["id"] == str(id_old).zfill(3))
        ]

    if len(df_new_jobs_i) == 1:
        row_j = df_new_jobs_i.iloc[0]

        action_j = row_j["action"]

        bool_0 = action_j == 'Job is busy, will skip'
        bool_1 = action_j == 'Time out or failed | Restarting isif 3 calc'
        bool_2 = action_j == 'Time out or failed | Restarting isif 7 calc'
        bool_3 = action_j == 'Job done | ISIF 3 done | --> isif 2'
        
        if bool_0 or bool_1 or bool_2 or bool_3:
            new_column_values_dict["running"] = True
        else:
            new_column_values_dict["running"] = False

        bool_0 = action_j == "Error, need manual attention"
        bool_1 = action_j == "Couldn't figure out what to do"
        if bool_0 or bool_1:
            new_column_values_dict["error"] = True

    elif len(df_new_jobs_i) == 0:
        new_column_values_dict["running"] = np.nan
    elif len(df_new_jobs_i) > 1:
        # print(df_new_jobs_i)
        # display(df_new_jobs_i)
        new_column_values_dict["running"] = "TEMP, more than 1 sys"

    # #########################################################################
    for key, value in new_column_values_dict.items():
        row_i[key] = value
    return(row_i)

df_i = df_static
df_static = df_i.apply(method, axis=1)

697 total AB2  structures in df_static

------------------------------------
87 systems with num_atoms >= 100

132 systems with num_atoms >= 75

In [9]:
697 # Total AB2
565 # AB2 | < 75 atoms
105 # AB2 | < 75 atoms | Not  done
# 32 # AB2 | < 75 atoms | Not  done | Not ignored
18 # AB2 | < 75 atoms | Not  done | Not ignored


# #####################################
14  # Currently running
7  # That have identified error

7

In [10]:
df = df_static.drop("static_id", axis=1)
df = df[
    (df["stoich"] == stoich_i) & \
    (df["num_atoms"] < 75) & \
#     (df["num_atoms"] > 75) & \
#     (df["num_atoms"] < 100) & \
#     (df["being_processed"] == False) & \
    (df["done"] == False) & \
    (df["ignored"] == False) & \
#     (df["running"] == True) & \
    [True for i in range(len(df))]
    ]
print("df.shape:", df.index.unique().shape)
# display(df)



from running_ids import sherlock_ids_running, slac_ids_running


slac_ids_running

print(len([i for i in df["id_old"].tolist() if str(i).zfill(3) in sherlock_ids_running]))
print(len([i for i in df["id_old"].tolist() if str(i).zfill(3) in slac_ids_running]))

# sherlock_ids_running
# df["id_old"].tolist()

tmp_list_0 = np.sort(
[str(i).zfill(3) for i in df["id_old"].tolist() if str(i).zfill(3) not in sherlock_ids_running + slac_ids_running]
)
print(tmp_list_0)

df.shape: (18,)
0
5
['060' '089' '118' '155' '193' '236' '250' '275' '303' '483' '587' '618'
 '649']


In [11]:
df_tmp = df_new_jobs[
    (df_new_jobs["id"].isin(tmp_list_0)) & \
    (df_new_jobs["stoich"] == "AB2")
    ].sort_values("id")
display(df_tmp); print("")

df = df_static
df_static[
    (df["id_old"].isin(tmp_list_0)) & \
    (df["stoich"] == "AB2") & \
    (df["error"] != True)
    ].sort_values("id_old")

,action,id,num_completed_isif_3,num_revisions,pre_path,source,stoich
6,"Error, need manual attention",060,3,8,/nfs/slac/g/suncatfs/flores12/PROJ_irox_ml_oer...,slac,AB2
47,"Error, need manual attention",089,2,5,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2
64,"Error, need manual attention",118,1,4,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2
130,"Error, need manual attention",236,2,8,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2
139,"Error, need manual attention",250,1,4,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2
247,"Error, need manual attention",483,3,6,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2
284,ALL DONE! | ISIF 2,587,3,21,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2
94,ALL DONE! | ISIF 2,618,3,9,/nfs/slac/g/suncatfs/flores12/PROJ_irox_ml_oer...,slac,AB2
107,Couldn't figure out what to do,649,0,16,/nfs/slac/g/suncatfs/flores12/PROJ_irox_ml_oer...,slac,AB2


,being_processed,cluster,done,error,id_old,ids_to_run,ignored,num_atoms,running,static_id,stoich
id_unique,,,,,,,,,,,
bgvr62z2ce,False,NaN,False,NaN,155,False,False,30,NaN,sarebilo_71,AB2
cqbax4mr9q,False,NaN,False,NaN,193,False,False,9,NaN,veliwepe_45,AB2
8tx2vung6y,False,NaN,False,NaN,275,False,False,24,NaN,tugekato_56,AB2
6amrb4m49u,False,NaN,False,NaN,303,False,False,12,NaN,watifago_85,AB2
9e6evkclch,True,sherlock,False,NaN,587,True,False,72,False,tabihoki_98,AB2
9pbu9hzwct,True,NaN,False,NaN,618,False,False,54,False,tuvigeno_32,AB2


In [12]:
# 60,
# 89,
# 118,
155,
193,
# 236,
# 250,
275,
303,
# 483,
# 649

(303,)

In [13]:
repeated_ids_list = []
for i in ids_master_list:
    if ids_master_list.count(i) > 1:
        repeated_ids_list.append(i)


set(repeated_ids_list)

NameError: name 'ids_master_list' is not defined